# Αναγνώριση Προτύπων
## Bonus Assignment , Κώστας Μαυρογιαννάκης 9789

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [2]:
# διαβάζουμε το αρχείο
with open("Dataset.csv") as file_name:
    array = np.loadtxt(file_name, delimiter=",")
# το σπάμε στα Χ,Υ
X_train=array[:,0:2]
Y_train=array[:,2]
# αλλάζουμε κλίμακα
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
# stack για μετα
XY_train=np.column_stack((X_train,Y_train))

In [3]:
names = [
    "Nearest Neighbors 3",
    "Nearest Neighbors 5",
    "Linear SVM",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bayes",
]
# τα πήρα έτοιμα από την sklearn
classifiers = [
    KNeighborsClassifier(3),
    KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
]

In [4]:
max=0
# κάνουμε iterate για διαφορετικό test set κάθε φορά ανάλογα την resample
for gamma in (np.arange(0.1,0.7,0.1)):
    # για να δοκιμάσω τους classifiers φτιάχνω test sets μέσω bootstrap
    XY_resample= resample(XY_train,n_samples=int(XY_train.shape[0]*gamma) )
    Y_test=XY_resample[:,2]
    X_test=XY_resample[:,:2]
    # iterate για τον καθε classifier
    for name,clf in zip(names,classifiers):
        # fit predict
        clf.fit(X_train,Y_train)
        Y_pred=clf.predict(X_test)
        accuracy = metrics.accuracy_score(Y_test, Y_pred)
        #print(accuracy,name)
        # αν έχει το μεγαλύτερο accuracy τον κρατάμε
        if(accuracy>max):
            max=accuracy
            bestclf=clf
            bestname=name

In [5]:
print("The highest accuracy is:",max,"with",bestname,"classifier")

The highest accuracy is: 0.9375 with Nearest Neighbors 3 classifier


In [ ]:
# ανοιγουμε τα άλλα 2 αρχεία και μετράμε το accuracy
with open("DatasetVal.csv") as file_name:
    X_Val = np.loadtxt(file_name, delimiter=",")
Y_predVal = bestclf.predict(X_Val)

In [ ]:
with open("Labels.csv") as file_name:
    Y_true = np.loadtxt(file_name, delimiter=",")

In [ ]:
print("My accuracy with",bestname,"classifier is:",metrics.accuracy_score(Y_true, Y_predVal))